# Table of Content

1. [Read in Data as Pandas DataFrame](#Read-in-Data-as-Pandas-DataFrame)
2. [Random Sampling](#Random-Sampling)
3. [Renaming Axis Indexes](#Renaming-Axis-Indexes)
4. [Indexing, Selecting and Filtering DataFrame](#Indexing,-Selecting-and-Filtering-DataFrame) 

**Combining and Merging Data Sets**

5.  [Concatenating Along an Axis](#Concatenating-Along-an-Axis)
6. [Database-style DataFrame Merges](#Database-style-DataFrame-Merges)
7. [Joining/ Merging on Index](#Joining/-Merging-on-Index)

**Reshaping and Pivoting**
8. [Reshaping by Melt](#Reshaping-by-Melt)
9. [Reshaping by pivoting](#Reshaping-by-pivoting)

**Data Transformation**

10. [Dealing with datetime](#Dealing-with-datetime) 

9. [Removing Duplicates](#Removing-Duplicates)

10. [Transforming Data Using a Function or Mapping](#Transforming-Data-Using-a-Function-or-Mapping)

11. [Replacing Values/ Handling Missing Values](#Replacing-Values/-Handling-Missing-Values)

12. [Discretization and Binning](#Discretization-and-Binning)


[Exercises](#Exercises)

Materials are adopted from the 
- Python for Data Analysis by Wes McKinney 
- Pandas 0.24.2 documentation

In [ ]:
# import required packages
import os
import pandas as pd
import numpy as np

# Read in Data as Pandas DataFrame

In [ ]:
#print(os.getcwd())
    
weather_full = pd.read_csv('../data/weather_description.csv')  #(45253, 37)
temp_full = pd.read_csv('../data/temperature.csv') # (45253, 37)
city_full = pd.read_csv('../data/city_attributes.csv') #(36, 4)

In [ ]:
weather_full.head(5)

In [ ]:
temp_full.head(5)

In [ ]:
city_full.head(5)

# Random Sampling

We might want to sample a subset of the entire dataset to do data wrangling & exploration. This is especially useful when working with large data sets.  

To select a random subset without replacement, we can 
1. slice off the first k rows of the dataframe, using `pandas.DataFrame.iloc` indexing 
2. randomly sampling k rows from the dataframe, using `pandas.DataFrame.sample`

In [ ]:
k = 1000

temp_small = temp_full.iloc[0:k]
temp_small = temp_full.sample(n=k, replace=False, random_state=0)

temp_small.head(2)

----

### Renaming Axis Indexes

In [ ]:
df = city_full.set_index('City')
df.head(2) 

In [ ]:
df2 = df.copy()

# Renaming Axis Indexes
df2.index.map(str.upper)

# assign to index, modifying the DataFrame in place
df2.index = df2.index.map(str.upper)

# using pandas.rename
df2.rename(index=str.title, columns=str.upper)

df2.rename(index={'San Francisco': 'SF', 'New York': 'NYC'},
           columns={'Country': 'Nation' })

# Indexing, Selecting and Filtering DataFrame

- `DataFrame.filter`: Subset rows or columns of dataframe according to labels in the specified index.

- `DataFrame.loc`: Access a group of rows and columns by label(s) or a boolean array.
- `DataFrame.iloc`: Purely integer-location based indexing for selection by position.

- `DataFrame.query`: Query the columns of a DataFrame with a boolean expression.


Pandas Doc: 
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.filter.html
- https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html#pandas.DataFrame.query


#### `DataFrame.filter(items=[list-like], like=[string], regex=[string (regular expression)], axis=[int/ string axis name])`

In [ ]:
# select columns by name

df.filter(items=['Latitude', 'Longitude'])
df.filter(regex='tude$', axis=1)

In [ ]:
# select rows 

df.filter(items=['Portland', 'Seattle'], axis=0)

df.filter(like='San', axis=0)
df.filter(regex='^San', axis=0)

df.filter(regex='land$', axis=0)

#### `DataFrame.iloc[<selection>, <selection>]`

In [ ]:
city_full.head(2)

In [ ]:
# Single selections using iloc and DataFrame

# Rows:
city_full.iloc[0] # first row of data frame - Note a Series data type output.
city_full.iloc[1] # second row of data frame 
city_full.iloc[-1] # last row of data frame

# Columns:
city_full.iloc[:,0] # first column of data frame 
city_full.iloc[:,1] # second column of data frame 
city_full.iloc[:,-1] # last column of data frame 

In [ ]:
# Multiple columns and rows can be selected together using the .iloc indexer.

# Multiple row and column selections using iloc and DataFrame
city_full.iloc[0:5] # first five rows of dataframe
city_full.iloc[:, 0:2] # first two columns of data frame with all rows
city_full.iloc[[0,3,6,24], [2,3]] # 1st, 4th, 7th, 25th row + 3nd, 4rd columns.
city_full.iloc[0:5, 1:] # first 5 rows and every columns from the 2nd of data frame.


#### `DataFrame.loc[<selection>, <selection>]`

In [ ]:
# Conditional that returns a boolean Series
city_full.loc[city_full['Country'] == 'Canada']

In [ ]:
# ... with column labels specified
city_full.loc[city_full['Country'] == 'Canada', ['Latitude', 'Longitude']]

In [ ]:
# ... that match multiple row values
city_full.loc[city_full['City'].isin(['New York', 'Boston'])]

In [ ]:
# ... that match row values on different columns 
city_full.loc[city_full['City'].str.endswith("land") & city_full['Country'].str.startswith("United")] 

In [ ]:
# ... 
city_full.loc[(city_full['Latitude'] > 40) & (city_full['Longitude'] <= -40)] 

In [ ]:
# A lambda function that yields True/False values can also be used.
city_full.loc[city_full['City'].apply(lambda x: len(x.split(' ')) == 2)] 

In [ ]:
# Selections can be achieved outside of the main .loc for clarity:
# Form a separate variable with your selections:
idx = city_full['City'].apply(lambda x: len(x.split(' ')) == 2)

# Select only the True values in 'idx' and the columns specified:
city_full.loc[idx, ['City']] #.reset_index(drop=True)

In [ ]:
# Does it return Pandas Series or DataFrame

print(type(city_full.loc[city_full['Country'] == 'Canada', 'City']))  # type Series when only one columns is selected
print(type(city_full.loc[city_full['Country'] == 'Canada', ['City']])) # DataFrame when list selection is used

#### `DataFrame.query[<expr>]`

In [ ]:
city_full.query('Country == "Canada" & Longitude < - 100')

----

# Combining and Merging Data Sets 

Data contained in `pandas` objects can be combined together in a number of built-in ways:
- `pandas.merge` connects rows in DataFrames based on one or more keys. 
- `pandas.join` Join columns of another DataFrame.
- `pandas.concat` glues or stacks together objects along an axis.
- `combine_first` instance method enables splicing together overlapping data to fill
in missing values in one object with values from another

Pandas doc: https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

### Concatenating Along an Axis 

- `pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, copy=True)`

In [ ]:
df1= temp_full.set_index('datetime').rename_axis(None)
df2 = weather_full.set_index('datetime').rename_axis(None)

result = pd.concat([df1, df2])
result.head()

### Database-style DataFrame Merges

- Merge or join operations combine data sets by linking rows using one or more keys 
- `pd.merge(left, right, how='merge method', on='key', left_on='left_key', right_on='right_key')`, 


Merge method |	SQL Join Name	| Description 
- | - | -
left	| LEFT OUTER JOIN	| Use keys from left frame only
right	| RIGHT OUTER JOIN	| Use keys from right frame only
outer	| FULL OUTER JOIN	| Use union of keys from both frames
inner	| INNER JOIN	    | Use intersection of keys from both frames
         
Pandas doc: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html         

In [ ]:
temp_small = temp_full.iloc[0:k]

temp_small_t = temp_small.set_index('datetime').rename_axis(None).transpose().rename_axis('City').reset_index()


In [ ]:
city_temp = pd.merge(city_full, temp_small_t, on='City')

city_temp.head()

#### Checking for duplicate keys

Users can use the validate argument to automatically check whether there are unexpected duplicates in their merge keys

In [ ]:
left = pd.DataFrame({'A' : [1,2], 'B' : [1, 2]})
right = pd.DataFrame({'A' : [4,5,6], 'B': [2, 2, 2]})
    
left.head(5)

In [ ]:
right.head(5)

In [ ]:
#result = pd.merge(left, right, on='B', validate="one_to_one")

### Joining/ Merging on Index 

- `DataFrame.join(other, on=None, how='left', lsuffix='', rsuffix='', sort=False)`

In [ ]:
# city_full.set_index('City', inplace=True)
# temp_small_t.set_index('City', inplace=True)

city_full.head()

In [ ]:
temp_small_t.head()

In [ ]:
city_temp = city_full.join(temp_small_t, on='City')
city_temp.head()

# same as
# result = pd.merge(city_full, temp_small_t, left_index=True, right_index=True, how='left')
# result.head(5)

# Reshaping and Pivoting 

In [ ]:
city_full.head()

### Reshaping by Melt

- `pandas.melt()`: Unpivots a DataFrame from wide format to long format, optionally leaving identifier variables set.
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.melt.html#pandas.DataFrame.melt

In [ ]:
city_long = pd.melt(city_full, id_vars = ['Country', 'City']) # var_name='Lat/Long', value_name='value'

# or, 
# city_full.melt(id_vars=['Country', 'City'], var_name='quantity')

### Reshaping by pivoting 

- `pandas.pivot` : Pivot a DataFrame from long to wide format by given index / column values.
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot.html

In [ ]:
city_wide = city_long.pivot(index='City', columns='variable', values='value')
city_wide.head()

# Data Transformation 

### Dealing with datetime

In [ ]:
weather_full['datetime'] = pd.to_datetime(weather_full['datetime'])

print(weather_full['datetime'].dtype)

In [ ]:
weather_full.head()

In [ ]:
import datetime

start_datetime = datetime.datetime(2012,10,1, 9, 0)
end_datetime = datetime.datetime(2012,10,1, 17, 0)


weather_full[weather_full.datetime.between(start_datetime, end_datetime)]


### Removing Duplicates

- `DataFrame.drop_duplicates([subset, keep, …])`: Return DataFrame with duplicate rows removed, optionally only considering certain columns.

In [ ]:
city_full.head()

In [ ]:
city_full.duplicated()

In [ ]:
df = pd.DataFrame([['Vancouver', 'Canada', 49.249660, -123.119339],
                  ['Portland', 'United States', 45.523449, -122.676208]],
                 columns = ['City', 'Country', 'Latitude', 'Longitude'])
city_full2 = city_full.append(df)
city_full2

In [ ]:
city_full2.duplicated()

In [ ]:
city_full2 = city_full2.drop_duplicates()
city_full2

In [ ]:
df = pd.DataFrame([['Vancouver', 'canada', 49.249660, -123.119339],
                  ['Portland', 'United States', 45.523449, -122.676208]],
                 columns = ['City', 'Country', 'Latitude', 'Longitude'])
city_full2 = city_full.append(df)
city_full2 = city_full2.drop_duplicates()
city_full2

In [ ]:
city_full2.drop_duplicates(['Country'])

### Transforming Data Using a Function or Mapping

In [ ]:
country_to_name_len = {
    'United States': 'twelve',
    'Canada': 'six',
    'Israel': 'six'
}
city_full['Name Length'] = city_full['Country'].map(country_to_name_len)
city_full

### Replacing Values/ Handling Missing Values

- `DataFrame.dropna([axis, how, thresh, …])`: Remove missing values.
- `DataFrame.fillna([value, method, axis, …])`:	Fill NA/NaN values using the specified method.
- `DataFrame.replace([to_replace, value, …])`:	Replace values given in to_replace with value.

In [ ]:
city_full2 = city_full.replace('twelve', 12)
city_full2 = city_full2.replace('six', 6)
city_full2

city_full2 = city_full2.replace(['twelve', 'six'], [12, 6])
city_full2

In [ ]:
temp_full.head()

In [ ]:
temp_full.replace(np.nan, 'no data')

In [ ]:
temp_full.fillna(0)

In [ ]:
na_values = {'Vancouver': 0, 'Portland': 1} #Replace NA's in each column with different values
temp_full.fillna(value = na_values)

In [ ]:
temp_full.fillna(value=na_values, limit=1) #replace only the first NA

In [ ]:
temp_full.dropna() #drop rows with at least one element missing

In [ ]:
temp_full.dropna(axis = 'columns') #drop columns with at least one element missing

In [ ]:
temp_full.dropna(how = 'all') #drop rows where all elements are missing
temp_full.dropna(thresh = 2) #drop rows with at least 2 non-NA values
temp_full.dropna(subset = ['Vancouver', 'Portland']) #drop rows where elements are missing in any columns specified

### Discretization and Binning

- `pandas.cut` Bin values into discrete intervals.

In [ ]:
bins = list(range(230, 330, 5))

temp_full2 = pd.read_csv('../data/temperature.csv')

temp_full2['Vancouver'] = pd.cut(temp_full2['Vancouver'], bins)
temp_full2['Portland'] = pd.cut(temp_full2['Portland'], bins)
temp_full2['San Francisco'] = pd.cut(temp_full2['San Francisco'], bins)
temp_full2['Seattle'] = pd.cut(temp_full2['Seattle'], bins)
temp_full2['Los Angeles'] = pd.cut(temp_full2['Los Angeles'], bins)
temp_full2

# Exercises

### Exercise 1


Output a dataframe with the temperatures in year 2013 of the cities in Canada.

Hint 1: we would need to transpose the `temp_full` table 

Hint 2: Information of country & cities is in `city_full` while information temperature & cities is in `temp_full`, hence we would need need to merge these two tables before filtering for `Country == "Canada"`


In [ ]:
##  Write your code here:

### Exercise 2

Highest temprature in each cities between year 2013 and 2014.

Hint 1: `index.dt.month in [6, 7, 8, 9]`, `index.dt.year in [2013, 2014]")` in `pd.query`
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.year.html
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.month.html

Hint 2: use `Pandas.max`: http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.max.html

In [ ]:
#  Write your code here: